In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import pandas as pd
from datetime import datetime
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
import warnings
warnings.filterwarnings("ignore")
%matplotlib inline
%config InlineBackend.figure_format='retina'

sns.set(style='whitegrid', palette='muted', font_scale=1.5)

rcParams['figure.figsize'] = 16, 10

C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passi

In [2]:
df= pd.read_csv("C:/Users/RAHAT/Downloads/Untitled Folder/Sales_f/data/Splited Data s-1 to s-5/STEST/S_1__I_582865.csv",parse_dates=['date'],index_col='date')

#convert date field from string to datetime
#df['date'] = pd.to_datetime(df['date'])

In [3]:
df.drop(['id','store_nbr','item_nbr','onpromotion'], axis=1,inplace=True)
df=df[0:300]
train_size = int(len(df) * 0.7)
test_size = len(df) - train_size
train, test = df.iloc[0:train_size], df.iloc[train_size:len(df)]
print(len(train), len(test))

210 90


In [4]:
def create_dataset(X, y, time_steps=1):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

In [5]:
time_steps = 20

# reshape to [samples, time_steps, n_features]

X_train, y_train = create_dataset(train, train.unit_sales, time_steps)
X_test, y_test = create_dataset(test, test.unit_sales, time_steps)

print(X_train.shape, y_test.shape)

(190, 20, 1) (70,)


In [6]:
model = keras.Sequential()
model.add(keras.layers.LSTM(
  units=128,
  input_shape=(X_train.shape[1], X_train.shape[2])
))
model.add(keras.layers.Dense(units=1))
model.compile(
  loss='mean_squared_error',
  optimizer=keras.optimizers.Adam(0.001)
)

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [7]:
history = model.fit(
    X_train, y_train,
    epochs=70,
    batch_size=16,
    validation_split=0.1,
    verbose=1,
    shuffle=False
)

Train on 171 samples, validate on 19 samples
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/70
171/171 [==============================] - 2s 9ms/sample - loss: 153.1093 - val_loss: 79.8527
Epoch 2/70
171/171 [==============================] - 0s 3ms/sample - loss: 69.5580 - val_loss: 28.6561
Epoch 3/70
171/171 [==============================] - 0s 3ms/sample - loss: 32.0959 - val_loss: 14.1904
Epoch 4/70
171/171 [==============================] - 0s 2ms/sample - loss: 22.9787 - val_loss: 14.4662
Epoch 5/70
171/171 [==============================] - 0s 3ms/sample - loss: 22.7490 - val_loss: 14.8340
Epoch 6/70
171/171 [==============================] - 0s 2ms/sample - loss: 22.5975 - val_loss: 14.2499
Epoch 7/70
171/171 [==============================] - 0s 3ms/sample - loss: 22.4459 - val_loss: 13.9631
Epoch 8/70
171/171 [==============================] - 0s 3ms/sample - loss: 22.4435 - val_loss: 13.9255
Epoch 9/70
171/171 [========

In [8]:
y_pred = model.predict(X_test)

In [9]:
from sklearn.metrics import mean_squared_error 
mean_squared_error(y_test,y_pred)

22.826047683344928